In [1]:
pip install pycuda

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 4.6 MB/s 
     |████████████████████████████████| 75 kB 4.5 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627871 sha256=e4eeec83829353b769247809795d51a7e1f5a2ae214d14c10c581d7262de36c5
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=3a8ba28d439136d62f21ed9122251d5f6714b6878fbc735d607c4d4915a9475a
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy
import time
import math

In [ ]:
def formatting(a,b): #needed only once
    try:
        x = len(a[0][0][0])
        for i in range(len(a)):
            for j in range(len(a[0])):
                for k in range(len(a[0][0])):
                    for l in range(len(a[0][0][0])):
                        b[l][k][i][j] = a[i][j][k][l]

    except:
        for i in range(len(a)):
            for j in range(len(a[0])):
                for k in range(len(a[0][0])):
                    b[k][i][j] = a[i][j][k]
    return b

In [ ]:
def deformatting(X,Y): #needed only once
    for i in range(len(X[0])):
        for j in range(len(X[0][0])):
            for k in range(len(X)):
                Y[i][j][k] = X[k][i][j]
    return Y

## Parallel sum of arrays

In [ ]:
def kernel_for_sum(arr1,arr2):

    h = numpy.int32(len(arr1))
    w = numpy.int32(len(arr1[0]))

    grid_h = math.ceil(h/32)
    grid_w = math.ceil(w/32)

    arr1_d = cuda.mem_alloc(arr1.nbytes)
    arr2_d = cuda.mem_alloc(arr2.nbytes)

    cuda.memcpy_htod(arr1_d,arr1)
    cuda.memcpy_htod(arr2_d,arr2)

    mod = SourceModule("""
    __global__ void Sum_of_arrays(int *arr1_d, int *arr2_d, int w) {
            int row = blockIdx.y*blockDim.y + threadIdx.y;
            int col = blockIdx.x*blockDim.y + threadIdx.x;

            arr1_d[row*w + col] = arr1_d[row*w + col] + arr2_d[row*w + col];
        }
    """)

    Sum_of_arrays = mod.get_function("Sum_of_arrays")
    Sum_of_arrays(arr1_d,arr2_d,w,block=(32,32,1),grid=(grid_w,grid_h,1))

    cuda.memcpy_dtoh(arr1,arr1_d)
    return arr1

## Parallel Convolution

In [ ]:
def convolution_single_step(tempinput, tempfilter, tempoutput):

    mod = SourceModule("""
__global__ void convolution_2d(int *input, int *filter, int *output, int height, int width){
    int FILTER_DIM = 3;
    int FILTER_OFFSET = FILTER_DIM/2 ;

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int start_r = row - FILTER_OFFSET;
    int start_c = col - FILTER_OFFSET;

    int temp = 0;
    for (int i = 0; i < FILTER_DIM; i++){
        for (int j = 0; j < FILTER_DIM; j++){
            if ((start_r + i) >= 0 && (start_r + i) < height){
                if ((start_c + j) >= 0 && (start_c + j) < width){
                    temp += input[(start_r + i) * width + (start_c + j)] * filter[i * FILTER_DIM + j];
                }
            }
        }
    }
    output[row * width + col] = temp;
}
""")
    height = numpy.int32(h)
    width = numpy.int32(w)
    
    gridHeight = math.ceil(h/32)
    gridWidth = math.ceil(w/32)

    #start = time.time()
    conv = mod.get_function("convolution_2d")

    #diff = 0

    tempinput = tempinput.astype(numpy.int32)
    tempfilter = tempfilter.astype(numpy.int32)
    tempoutput = tempoutput.astype(numpy.int32)

    tempinputD = cuda.mem_alloc(tempinput.nbytes)
    tempfilterD = cuda.mem_alloc(tempfilter.nbytes)
    tempoutputD = cuda.mem_alloc(tempoutput.nbytes)

    cuda.memcpy_htod(tempinputD, tempinput)
    cuda.memcpy_htod(tempfilterD, tempfilter)
    cuda.memcpy_htod(tempoutputD, tempoutput)

    #startker = time.time()
    conv(tempinputD, tempfilterD, tempoutputD, height, width, block=(4,4,1), grid=(gridWidth,gridHeight,1))  # block format is important! (width,height,1)
    #endker = time.time()

    #differencekernel = endker-startker
    #diff += differencekernel

    cuda.memcpy_dtoh(tempoutput, tempoutputD)

    #end = time.time()

    return tempoutput


In [ ]:
def convolution_layer(X,W):
    (channels, height_X, width_X) = X.shape
    
    (filters, channels, height_W, width_W) = W.shape
    
    final_output = []
    for f in range(filters):
        base_array = numpy.zeros(shape=(height_X,width_X)).astype(numpy.int32)
        final_output.append([])
        for c in range(channels):   
            tempoutput = numpy.zeros(shape=(height_X,width_X))
            one_channel_output = convolution_single_step(X[c], W[f][c], tempoutput)   
            one_kernel_output = kernel_for_sum(base_array,one_channel_output)            
            base_array = one_kernel_output
        final_output[-1] = one_kernel_output
    return numpy.array(final_output)

In [ ]:
h = 400
w = 288
ch = 32
filters = 1
W_w = W_h = 3

X = numpy.random.randint(1,5,size=(ch,h,w))
X = X.astype(numpy.int32)

W = numpy.random.randint(1,3,size=(filters,ch,W_h,W_w))
W = W.astype(numpy.int32)


#print(X)
#print(W)
timeStart = time.time()
s = convolution_layer(X, W)
timeEnd = time.time()
#print(s)
print(numpy.shape(s))
print(timeEnd-timeStart)

(1, 400, 288)
0.10384821891784668


**PARALLEL MAXPOOLING**

In [41]:
def Max_Pool_Single_Step(tempinput,tempoutput):
    widthO = w/2
    heightO = h/2
    widthO = numpy.int32(widthO)
    heightO = numpy.int32(heightO)
    mod = SourceModule("""
    __global__ void maxpool(int *input, int *output, int inputDimX, int inputDimY, int outputDimX, int outputDimY){

        int row = blockIdx.y * blockDim.y + threadIdx.y;
        int col = blockIdx.x * blockDim.x + threadIdx.x;

        if((row<outputDimY)&&(col<outputDimX)){
            int temp[4];
            temp[0] = input[(2*row) * inputDimX + (2*col)];
            temp[1] = input[(2*row) * inputDimX + (2*col+1)]; 
            temp[2] = input[(2*row+1) * inputDimX + (2*col)]; 
            temp[3] = input[(2*row+1) * inputDimX + (2*col+1)];

            int i, max = 0;
            for(i=0; i<4; i++){
                if(temp[i]>max){
                    max = temp[i];
                }
            }
            output[row * outputDimX + col] = max;
        }
    }
    """)
    gridHeight = math.ceil(h/64)
    gridWidth = math.ceil(w/64)

    diff = 0

    start = time.time()
    Mpool = mod.get_function("maxpool")


    tempinput = tempinput.astype(numpy.int32)
    tempoutput = tempoutput.astype(numpy.int32)

    tempinputD = cuda.mem_alloc(tempinput.nbytes)
    tempoutputD = cuda.mem_alloc(tempoutput.nbytes)

    cuda.memcpy_htod(tempinputD, tempinput)
    cuda.memcpy_htod(tempoutputD, tempoutput)

    startker = time.time()
    Mpool(tempinputD, tempoutputD, width, height, widthO, heightO, block=(32,32,1), grid=(gridWidth,gridHeight,1))  # block format is important! (width,height,1)
    endker = time.time()
    
    differencekernel = endker-startker
    diff += differencekernel

    cuda.memcpy_dtoh(tempoutput, tempoutputD)
    return tempoutput

In [42]:
def Max_pool_layer(X):
    (channels,height,width) = X.shape
    widthO = w/2
    heightO = h/2
    widthO = numpy.int32(widthO)
    heightO = numpy.int32(heightO)
    final_output = []
    for c in range(channels):
        output = numpy.zeros(shape=(heightO,widthO)).astype(numpy.int32)
        one_channel_output = Max_Pool_Single_Step(X[c],output)
        final_output.append(one_channel_output)
    return numpy.array(final_output)

In [47]:
h = 64
w = 64
c = 1


X = numpy.random.randint(0,10,size=(c,h,w)).astype(numpy.int32)
height = numpy.int32(h)
width = numpy.int32(w)
channel = numpy.int32(c)

print(X)
m = Max_pool_layer(X)
print("----------")
print(m)
print(numpy.shape(m))

[[[9 7 8 ... 5 0 5]
  [6 2 1 ... 1 6 6]
  [3 5 2 ... 9 2 8]
  ...
  [0 2 5 ... 2 7 5]
  [6 1 1 ... 6 2 2]
  [2 6 7 ... 5 0 4]]]
----------
[[[9 9 6 ... 8 5 6]
  [9 5 9 ... 6 9 8]
  [9 7 8 ... 9 9 6]
  ...
  [8 9 9 ... 1 9 8]
  [4 8 8 ... 8 8 7]
  [6 8 6 ... 7 6 4]]]
(1, 32, 32)


**CONVOLUTION TRANSPOSE**

In [ ]:
def conv2DTranspose(tempinput, tempfilter, tempoutput):
    mod = SourceModule("""
    __global__ void convTranspose(int *input, int *filter, int *output, int inputDimX, int inputDimY, int outputDimX){

    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int i, j, temp[4];
    for(i = 0; i < 2; i++){
        for(j = 0; j < 2; j++){
            if((row<inputDimY)&&(col<inputDimX)){
                output[(2*row + i) * outputDimX + (2*col + j)] = input[row * inputDimX + col] * filter[2*i + j];
            }
        }
    }
    //output[(2*row) * outputDimX + (2*col)] = temp[0];
    //output[(2*row) * outputDimX + (2*col+1)] = temp[1];
    //output[(2*row+1) * outputDimX + (2*col)] = temp[2];
    //output[(2*row+1) * outputDimX + (2*col+1)] = temp[3];

    }
    # """)
    gridHeight = math.ceil(h/32)
    gridWidth = math.ceil(w/32)

    wo = 2*w

    height = numpy.int32(h)
    width = numpy.int32(w)

    widthO = numpy.int32(wo)

    #diff = 0

    #start = time.time()
    convtrans = mod.get_function("convTranspose")

    tempinput = tempinput.astype(numpy.int32)
    tempfilter = tempfilter.astype(numpy.int32)
    tempoutput = tempoutput.astype(numpy.int32)

    tempinputD = cuda.mem_alloc(tempinput.nbytes)
    tempfilterD = cuda.mem_alloc(tempfilter.nbytes)
    tempoutputD = cuda.mem_alloc(tempoutput.nbytes)

    cuda.memcpy_htod(tempinputD, tempinput)
    cuda.memcpy_htod(tempfilterD, tempfilter)
    cuda.memcpy_htod(tempoutputD, tempoutput)

    startker = time.time()
    convtrans(tempinputD, tempfilterD, tempoutputD, width, height, widthO, block=(32,32,1), grid=(gridWidth,gridHeight,1))  # block format is important! (width,height,1)
    endker = time.time()
    
    #differencekernel = endker-startker
    #diff += differencekernel

    cuda.memcpy_dtoh(tempoutput, tempoutputD)

    #end = time.time()

    return tempoutput


In [ ]:
def Conv2DTranspose_layer(X, W):
    global total_ch
    global total_soa
    (n_c_prev,height,width) = X.shape
    (filters,n_c_prev,f,f) = W.shape
    final_output = []
    for f in range(filters):
        base_array = numpy.zeros(shape=(height*2,width*2)).astype(numpy.int32)  #initializing with zero
        final_output.append([])
        for c in range(n_c_prev):
            start = time.time()
            tempoutput = numpy.zeros(shape=(height*2,width*2))
            one_channel_output =  conv2DTranspose(X[c], W[f][c], tempoutput)
            # print("input",X[c])
            # print("filter",W[f][c])
            # print("output",one_channel_output)
            end = time.time()
            total_ch+=(end-start)
            #print(np.array(one_channel_output).shape)
            start = time.time()
            # one_kernel_output = sum_of_arrays(base_array,one_channel_output)
            one_kernel_output = kernel_for_sum(base_array,one_channel_output)
            #print("channel output", one_kernel_output)
            end = time.time()
            #print("Time for one iteration:",end-start)
            total_soa+=(end-start)
            #print(np.array(one_kernel_output).shape)
            base_array = one_kernel_output
        final_output[-1] = one_kernel_output
    return numpy.array(final_output)

In [ ]:
h = 200
w = 144
ch = 64
filters = 32
W_w = W_h = 2

X = numpy.random.randint(1,5,size=(ch,h,w))
X = X.astype(numpy.int32)

W = numpy.random.randint(1,3,size=(filters,ch,W_h,W_w))
W = W.astype(numpy.int32)

#print(X)
#print(W)
total_ch = 0
total_soa = 0 #time taken by sum_of_arrays
s = Conv2DTranspose_layer(X, W)
#print(s)
print(numpy.shape(s))
print("time for convtrans: ",total_ch)
print("time for sum_of_array function: ",total_soa)
print(total_ch + total_soa)

(32, 400, 288)
time for convtrans:  1.5371813774108887
time for sum_of_array function:  2.993251085281372
4.530432462692261
